<h1><b>Mi primera red neuronal en TensorFlow</b> </h1>

<author>Julio Waissman Vilanova</author>

<br/>

<a target="_blank" href="https://colab.research.google.com/github/juliowaissman/intro-rn/blob/main/mnist-densa.ipynb">
<img src="https://i.ibb.co/2P3SLwK/colab.png" width=30pt />
<i>Para usar en Google Colab</i></a>

## Introducción

Vamos a hacer nuestra primera red neuronal en lo que se considera el *Hola Mundo* de las redes neuronales: el conjunto de datos [MNIST](http://yann.lecun.com/exdb/mnist/). El conjunto de datos MNIST consiste en 60,000 imagenes en blanco y negro de entrenamiento y 10,000 de prueba. Todas las imagenes son sobre dígitos escritos a mano. Así, la salida de la red neuronal debería ser la probabilidad de preteneces a 10 clases diferentes: los dígitos de 0 a 9.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

Vamos bajando el conjunto de datos y revisando unas imágenes. El modulo [`tf.keras.dataset`](https://www.tensorflow.org/api_docs/python/tf/keras/datasets) viene con algunos conjuntos muy populares:

In [ ]:
# TensorFlow ya te permite bajar ciertos conjuntos de datos famosos

mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = (np.expand_dims(train_images, axis=-1)/255.).astype(np.float32)
train_labels = (train_labels).astype(np.int64)
test_images = (np.expand_dims(test_images, axis=-1)/255.).astype(np.float32)
test_labels = (test_labels).astype(np.int64)

In [ ]:
plt.figure(figsize=(8,8))
random_inds = np.random.choice(60000,64)
for i in range(64):
    plt.subplot(8,8,i+1)
    plt.grid(False)
    plt.axis('off')
    image_ind = random_inds[i]
    plt.imshow(np.squeeze(train_images[image_ind]), cmap=plt.cm.binary)
    plt.title(train_labels[image_ind], y=.8)
plt.show()

## Diseño de una red neuronal densa


Construiremos una red neuronal simple que consta de dos capas completamente conectadas y la aplicaremos a la tarea de clasificación de dígitos. En última instancia, nuestra red generará una distribución de probabilidad entre las clases de 10 dígitos (0-9). Esta primera arquitectura que construiremos se muestra a continuación:

![](https://raw.githubusercontent.com/aamini/introtodeeplearning/master/lab2/img/mnist_2layers_arch.png "Arquitectura CNN para clasificación MNIST")

Para definir la arquitectura de esta red neuronal completamente conectada, usaremos la API de Keras y definiremos el modelo usando la clase [`Sequential`](https://www.tensorflow.org/api_docs/python/tf/keras /modelos/Secuencial).

Usamos primero una capa [`Flatten`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten), que aplana la entrada para que pueda introducirse en el modelo. Las capas densas, en la primera usaremos una activación no lineal conocida como `ReLU` y para la salida, como queremos una distribución de probabilidad, usaremos una activación tipo `softmax`.

En el siguiente bloque, definirá las capas completamente conectadas.

In [15]:
def build_fc_model():
  fc_model = tf.keras.Sequential([
      # Capa tipo Faltten para aplanar las imágenes
      tf.keras.layers.Flatten(),

      # '''TODO: Capa oculta completamente conectada.'''
      tf.keras.layers.Dense(128, activation= #'''TODO'''),

      # '''TODO: Define la capa de salida con las probabilidades por clase'''
      #[TODO Capa densa con las probabilidades de salida]

  ])
  return fc_model

model = build_fc_model()

A medida que avancemos en la siguiente parte, es posible que desees realizar cambios en la arquitectura definida anteriormente. **Ten en cuenta que para actualizar el modelo más adelante, deberás volver a ejecutar la celda anterior para reinicializar el modelo.**

Pensemos en la red que acabamos de crear. La primera capa de esta red, `tf.keras.layers.Flatten`, transforma el formato de las imágenes de una matriz 2D (28 x 28 píxeles) a un vector 1D de 28 * 28 = 784 entradas. Puede pensar en esta capa como desapilar filas de píxeles en la imagen y alinearlas. No hay parámetros aprendidos en esta capa; solo reformatea los datos.

Una vez aplanados los píxeles, la red consta de una secuencia de dos capas `tf.keras.layers.Dense`. Estas son capas neuronales completamente conectadas. La primera capa "densa" tiene 128 nodos (o neuronas). La segunda capa debe devolver una serie de puntuaciones de probabilidad que suman 1. Cada nodo contiene una puntuación que indica la probabilidad de que la imagen actual pertenezca a una de las clases de dígitos escritos a mano.

Eso define nuestro modelo totalmente conectado

## Compila el modelo

Antes de entrenar el modelo, necesitamos definir algunas configuraciones más. Estos se agregan durante el paso [`compile`](https://www.tensorflow.org/api_docs/python/tf/keras/models/Sequential#compile) del modelo:

* *Función de pérdida*: define cómo medimos la precisión del modelo durante el entrenamiento. Durante el entrenamiento queremos minimizar esta función, lo que "dirigirá" el modelo en la dirección correcta.
* *Optimizador*: define cómo se actualiza el modelo en función de los datos que ve y su función de pérdida.
* *Métricas*: aquí podemos definir las métricas utilizadas para monitorear los pasos de capacitación y prueba. En este ejemplo, veremos *accuracy*, la fracción de imágenes que están clasificadas correctamente.

Comenzaremos utilizando un optimizador de descenso de gradiente estocástico (SGD) inicializado con una tasa de aprendizaje de 0.1. Dado que estamos realizando una tarea de clasificación categórica, querremos utilizar la [pérdida de entropía cruzada](https://www.tensorflow.org/api_docs/python/tf/keras/metrics/sparse_categorical_crossentropy).

Mas adelante es posible que quieras experimentar tanto con la elección del optimizador como con la tasa de aprendizaje y evaluar cómo afectan la precisión del modelo entrenado.

In [16]:
model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=1e-1),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

## Entrena el modelo

Ahora estamos listos para entrenar nuestro modelo, lo que implicará introducir los datos de entrenamiento ("train_images" y "train_labels") en el modelo y luego pedirle que aprenda las asociaciones entre imágenes y etiquetas. También necesitaremos definir el tamaño del lote y la cantidad de épocas, o iteraciones sobre el conjunto de datos MNIST, que se usarán durante el entrenamiento.

En la libreta pasada, vimos cómo podemos usar `GradientTape` para optimizar las pérdidas y entrenar modelos con descenso de gradiente estocástico a pie.

Con Keras, después de definir la configuración del modelo y el optimizador en el paso `compilar`, también podemos realizar el entrenamiento llamando al metodo [`fit`](https://www.tensorflow.org/api_docs/python/tf/keras/models/Sequential#fit). Usaremos esto para entrenar nuestro modelo totalmente conectado.

In [ ]:
BATCH_SIZE = 64
EPOCHS = 5

model.fit(
    train_images,
    train_labels,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS
)

A medida que el modelo se entrena, se muestran las métricas de pérdida y precisión. Con cinco épocas y una tasa de aprendizaje de 0.01, este modelo completamente conectado debería alcanzar una precisión de aproximadamente 0.97 (o 97%) en los datos de entrenamiento.

## Evaluar la calidad del modelo entrenado

Ahora que hemos entrenado el modelo, podemos pedirle que haga predicciones sobre un conjunto de pruebas que no haya visto antes. En este ejemplo, la matriz `test_images` comprende nuestro conjunto de datos de prueba. Para evaluar la precisión, podemos verificar si las predicciones del modelo coinciden con las etiquetas de `test_labels`.

Vamos a utilizar el método [`evaluate`](https://www.tensorflow.org/api_docs/python/tf/keras/models/Sequential#evaluate) para evaluar el modelo en el conjunto de datos de prueba.

In [ ]:
test_loss, test_acc = model.evaluate(
    test_images,
    test_labels
)

print('Test accuracy:', test_acc)

Puedes observar que la precisión del conjunto de datos de prueba es un poco menor que la precisión del conjunto de datos de entrenamiento.

Si esta brecha entre la precisión del entrenamiento y la precisión de las pruebas es muy grande, entonces hay un *sobreajuste*, que es cuando un modelo de aprendizaje automático funciona peor con datos nuevos que con sus datos de entrenamiento. Así, aunque el modelo parezca bueno, es poco confiable en nuevos datos, que es para lo que lo queremos en ñultima instancia.

**¿Cuál es la mayor precisión que puede lograr con este primer modelo totalmente conectado?**

## Otra base de datos más compleja

Debido a que MNIST es un conjunto relativamente muy sencillo de ajustar, hay otros conjuntos propuestos ligeramente más complicados de solucionar, entre los que se encuentre el [Fashion-MNIST](https://github.com/zalandoresearch/fashion-mnist) que cambia los dígitos a mano por cosas ligeramiente más complicadas como tipos de ropa, zapatos y accesorios.

El tamaño de las imágenes, el número de clases, así como el número de ejemplos de entrenamiento y aprendizaje son igualitos a los de MNIST.

In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
train_images = (np.expand_dims(train_images, axis=-1)/255.).astype(np.float32)
train_labels = (train_labels).astype(np.int64)
test_images = (np.expand_dims(test_images, axis=-1)/255.).astype(np.float32)
test_labels = (test_labels).astype(np.int64)

In [ ]:
plt.figure(figsize=(8,8))
random_inds = np.random.choice(60000,64)
for i in range(64):
    plt.subplot(8,8,i+1)
    plt.grid(False)
    plt.axis('off')
    image_ind = random_inds[i]
    plt.imshow(np.squeeze(train_images[image_ind]), cmap=plt.cm.binary)
    plt.title(train_labels[image_ind], y=.9)
plt.show()

## Taller

**Diseña una red neuronal densa, selecciona los hiperparámetros y entrenala para el conjunto de datos de Fashion MNIST.**

**Tienes 15 minutos, y vamos a ver quien logra el mayor *accuracy* en el conjunto de test, y con el menor sobreajuste**

In [23]:
def build_fashon_model():
  model = tf.keras.Sequential([
      # Capa tipo Faltten para aplanar las imágenes
      tf.keras.layers.Flatten(),

      # '''TODO: Agrega cuantas capas y unidades quieras'''
      #[TODO Capa(s) densas oculta(s)],

      # '''TODO: Define la capa de salida con las probabilidades por clase'''
      #[TODO Capa densa con las probabilidades de salida]
  ])
  return model

model = build_fashon_model()

In [24]:
# Podrias cambiar el optimizador o la tasa de aprendizaje
lr = 1e-1
optimizador = tf.keras.optimizers.SGD(learning_rate=lr)

model.compile(
    optimizer=optimizador,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# Puedes probar con el tamaño de batch o los epochs
BATCH_SIZE = 64
EPOCHS = 5

model.fit(
    train_images,
    train_labels,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS
)

In [ ]:
# Aquí vamos a ver que tan ben funciona
test_loss, test_acc = model.evaluate(
    test_images,
    test_labels
)
print('Test accuracy:', test_acc)